In [2]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "21b6f0306d21c0cbb0bf50acb990de29-e596da2fe30f3295f91702e3928878ad"
ACCOUNT_ID = "101-004-27643856-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type":"application/json"
})

In [6]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [7]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params=None, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [11]:
data

{'instruments': [{'name': 'XAG_USD',
   'type': 'METAL',
   'displayName': 'Silver',
   'pipLocation': -4,
   'displayPrecision': 5,
   'tradeUnitsPrecision': 0,
   'minimumTradeSize': '1',
   'maximumTrailingStopDistance': '1.00000',
   'minimumTrailingStopDistance': '0.00050',
   'maximumPositionSize': '0',
   'maximumOrderUnits': '500000',
   'marginRate': '0.10',
   'guaranteedStopLossOrderMode': 'ALLOWED',
   'minimumGuaranteedStopLossDistance': '0.0800',
   'guaranteedStopLossOrderExecutionPremium': '0.02999999999998',
   'guaranteedStopLossOrderLevelRestriction': {'volume': '25000',
    'priceRange': '0.25'},
   'tags': [{'type': 'ASSET_CLASS', 'name': 'COMMODITY'},
    {'type': 'KID_ASSET_CLASS', 'name': 'METAL'},
    {'type': 'BRAIN_ASSET_CLASS', 'name': 'METAL'}],
   'financing': {'longRate': '-0.0692',
    'shortRate': '0.0471',
    'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
     {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'WEDNESDA

In [12]:
instruments_list = data['instruments']

In [13]:
len(instruments_list)

123

In [14]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [15]:
key_i = ['name', 'type', 'displayName', 'pipLocation',
         'displayPrecision', 'tradeUnitsPrecision']

In [16]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i}

In [17]:
instruments_dict['GBP_NZD']

{'name': 'GBP_NZD',
 'type': 'CURRENCY',
 'displayName': 'GBP/NZD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0}

In [18]:
with open("../data/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [19]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)    
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data 


def get_candles_df(data):
    if len(data) ==0:
        return pd.DataFrame() 
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['volume'] = candle['volume']
        new_dict['time'] = parser.parse(candle['time']) #show time better
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}-{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
        code,data = fetch_candles(pair_name, count, granularity)
        if code !=200:
            print("Failed", pair_name, data)
            return
        
        if len(data) == 0:
             print("No candles", pair_name)
        candles_df = get_candles_df(data) 
        candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
        print(f"{pair_name} {granularity}{candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
              
        

In [20]:
code, data = fetch_candles("GBP_NZD", count=10, granularity= "H4")
candles_df = get_candles_df(data)

In [21]:
create_data_file("GBP_NZD", count=10, granularity= "H4")

GBP_NZD H49 candles, 2023-12-11 06:00:00+00:00 2023-12-12 14:00:00+00:00


In [22]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [23]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity= g)

EUR_USD H14000 candles, 2023-04-24 02:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_USD H44000 candles, 2021-05-21 01:00:00+00:00 2023-12-12 14:00:00+00:00
EUR_GBP H14000 candles, 2023-04-24 02:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_GBP H44000 candles, 2021-05-21 01:00:00+00:00 2023-12-12 14:00:00+00:00
EUR_JPY H14000 candles, 2023-04-24 02:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_JPY H44000 candles, 2021-05-20 17:00:00+00:00 2023-12-12 14:00:00+00:00
EUR_CHF H14000 candles, 2023-04-24 02:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_CHF H44000 candles, 2021-05-21 01:00:00+00:00 2023-12-12 14:00:00+00:00
EUR_NZD H14000 candles, 2023-04-24 01:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_NZD H44000 candles, 2021-05-21 01:00:00+00:00 2023-12-12 14:00:00+00:00
EUR_CAD H14000 candles, 2023-04-24 02:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_CAD H44000 candles, 2021-05-21 01:00:00+00:00 2023-12-12 14:00:00+00:00
EUR_AUD H14000 candles, 2023-04-24 02:00:00+00:00 2023-12-12 18:00:00+00:00
EUR_AUD H440